In [5]:
import pandas as pd
from scipy.stats import zscore
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import neural_network

In [14]:
data = pd.read_csv("../backend/dataset/indian_water_physicochemical_parameters.csv", encoding = 'unicode_escape')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data['date'] = pd.to_datetime(data['date'])
data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2052.0,TUIRIAL UPPER CATCHMENT,MIZORAM,23.6,6.9,7.1,186.0,1.6,0.10,233.0,29.0,2014,2014-12-22
734,2053.0,TUIRIAL LOWER CATCHMENT,MIZORAM,23.5,6.8,7.1,201.0,1.4,0.20,233.0,86.0,2014,2014-12-24
735,1403.0,"GUMTI AT U/S SOUTH TRIPURA,TRIPURA",TRIPURA,27.3,6.2,7.8,165.0,2.1,0.00,282.0,340.0,2014,2014-12-25
736,1404.0,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",TRIPURA,29.1,6.4,7.8,158.0,4.1,0.50,458.0,500.0,2014,2014-12-26


In [27]:
from aquasentinel import predict_wqi, get_wqc 

In [38]:
specified_date = '2014-12-27'  

df = data.copy()
df.set_index('date', inplace=True)
# Convert the index type to string
df.index = df.index.astype(str)

# Get the data for the specified date
data_for_date = df.loc[specified_date]

df = df.drop(columns=["STATION CODE", "LOCATIONS", "STATE", "Temp", "year"])
for column in df.columns[:len(df.columns)- 1]:
            df[column] = pd.to_numeric(df[column], errors='coerce')
            
df.fillna(df.median(), inplace=True)
z_scores = df.apply(zscore)
outliers_count = (z_scores.abs() > 3).sum()
df_outliers = np.where(np.abs(z_scores) > 3)[0]
df_filtered = df.drop(df.index[df_outliers])

# Scale the dataset
scaler = StandardScaler()
X = df_filtered.values
X_scaled = scaler.fit_transform(X)
index_number = df_filtered.index.get_loc(specified_date)
index_number
wqi = predict_wqi(X_scaled[index_number])
print("Predicted WQI:", wqi)
        
wqc = get_wqc(wqi)
wqc


Shape of X_scaled: (659, 7)
(659, 7)
Predicted WQI: 95.78776501912307


c:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


'poor'

In [31]:
df_filtered


,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean
date,,,,,,,
2012-01-01,7.1,7.8,151.0,8.0,0.08,233.0,465.0
2012-01-04,7.2,7.4,319.0,0.7,0.18,5.0,10.0
2012-01-05,6.2,8.4,894.0,9.0,0.38,140.0,350.0
2012-01-07,5.9,7.2,52.0,2.1,0.08,233.0,90.0
2012-01-10,6.1,6.8,408.0,1.3,0.17,1300.0,3808.0
...,...,...,...,...,...,...,...
2014-12-22,6.9,7.1,186.0,1.6,0.10,233.0,29.0
2014-12-24,6.8,7.1,201.0,1.4,0.20,233.0,86.0
2014-12-25,6.2,7.8,165.0,2.1,0.00,282.0,340.0


In [39]:
df.describe()

,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean
count,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000,738.000000
mean,6.583198,7.290379,909.699187,3.694537,1.279350,2128.460705,3947.182927
std,1.063475,0.440368,2574.757652,6.421942,1.768243,6397.810887,11161.052236
min,2.400000,5.600000,30.000000,0.100000,0.000000,2.000000,0.000000
25%,6.100000,7.000000,74.250000,1.200000,0.250000,30.000000,67.000000
50%,6.700000,7.300000,144.000000,1.800000,0.545000,233.000000,420.000000
75%,7.200000,7.600000,352.750000,3.100000,1.580000,616.750000,1556.750000
max,9.900000,8.400000,18569.000000,88.000000,11.810000,53083.000000,87167.000000
